In [1]:
import numpy as np
import itertools
import datetime

import tensorflow as tf
import sklearn as sk

from gensim.models import Word2Vec

import Functions as fn
import Iterator as it
from DS import DS
from Set import pool
from FFModel import FF_Model
from RNNModel import RNN_Model

c:\users\luka\anaconda3\envs\tensorflow13\lib\site-packages\gensim\utils.py:865: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


# Data Loading

In [2]:
Dataset = pool()
Dataset.load_texts('raw_texts')
Dataset.load_labels('raw_labels')

Raw Text Load Complete
Raw Labels Load Complete


In [3]:
target_dict = fn.load_labels('final_meta/labels')

Label Load Complete


# Embedding Generation

In [4]:
train_set = pool(data=(Dataset.get_DS(stage='test', labelled='yes')).data[:-10])
validation_set = pool(data=(Dataset.get_DS(stage='test', labelled='yes')).data[-10:])
test_set = Dataset.get_DS(stage='train', labelled='yes')
set_1 = Dataset.get_DS(stage='train', labelled='no')
set_2 = Dataset.get_DS(stage='test', labelled='no')
set_1.append(set_2.data)
set_1.append(train_set.data)
emb_set = set_1
print(emb_set.size, train_set.size, validation_set.size, test_set.size)

4585 238 10 10


In [5]:
#emb_set.process_for_embedding()
#sentences = emb_set.get_sentences()
#fn.write_sentences(sentences, 'final_meta/sentences')
sentences = fn.load_sentences('final_meta/sentences')

#model = Word2Vec(sentences, min_count=1, size=100)
#model.save('final_meta/W2V')
model = Word2Vec.load('final_meta/W2V')

vocab = model.wv.vocab.keys()

Sentence Load Complete


# Layer and Index Loading

In [6]:
#word_indices, emb_layer = fn.get_index_and_emb_layer(model)
#fn.write_word_indices(word_indices, 's2s/word_indices')
#fn.write_emb_layer(emb_layer, 's2s/emb_layer')

word_indices = fn.load_word_indices('s2s/word_indices')
emb_layer = fn.load_emb_layer('s2s/emb_layer')

Word Indices Load Complete
Embedding Layer Load Complete


# RNN Testing

In [7]:
train_set.process_for_s2s_testing()
validation_set.process_for_s2s_testing()
test_set.process_for_s2s_testing()

In [8]:
entry = 19

print(test_set.data[0].enc_inputs[entry])
print(len(test_set.data[0].enc_inputs[entry]))
print(test_set.data[0].enc_labels[entry])
print(len(test_set.data[0].enc_labels[entry]))
print(test_set.data[0].dec_inputs[entry])
print(len(test_set.data[0].dec_inputs[entry]))
print(test_set.data[0].dec_outputs[entry])
print(len(test_set.data[0].dec_outputs[entry]))

['vancomycin', '<start>', 'her', 'graft', 'the', 'remainder', 'of', 'the', 'hospital', 'course', 'was', 'unremarkable', 'on', 'the', '<num>', 'of', 'july', ',', 'she', 'was', 'discharged', 'back', 'to', 'the', 'rose-walt', 'hospital', 'in', 'coln', 'discharge', 'medications', 'vancomycin', '<num>', 'mg', 'iv', 'q', 'd', ',', 'ofloxacin', '<num>', 'mg', 'po', 'bid', '(', 'both', 'antibiotics', 'to', 'continue', 'for', 'an', 'additional', 'two', 'week', 'course', ')', ',', 'coumadin', 'with', 'target']
58
[[1], [0], [6], [6], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [1], [2], [2], [3], [4], [4], [0], [1], [2], [2], [3], [4], [0], [0], [1], [0], [0], [5], [5], [5], [5], [5], [5], [0], [0], [1], [0], [0]]
58
['<go>', 'vancomycin', '<num>', 'mg', 'iv', 'q', 'd', 'for', 'an', 'additional', 'two', 'week', 'course']
13
['vancomycin', '<num>', 'mg', 'iv', 'q', 'd', 'for', 'an', 'additional', 'two', 'week', 

In [9]:
max_enc_inp = 0
max_dec_inp = 0
for setin in [train_set, validation_set, test_set]:
    for case in setin.data:
        for enc_inp  in case.enc_inputs:
            max_enc_inp = max(max_enc_inp, len(enc_inp))
        for dec_inp  in case.dec_inputs:
            max_dec_inp = max(max_dec_inp, len(dec_inp))
print(max_enc_inp, max_dec_inp)

97 34


In [10]:
sets = {}
sets['train'] = train_set.get_s2s_sets(word_indices, max_enc_inp, max_dec_inp)
sets['validation'] = validation_set.get_s2s_sets(word_indices, max_enc_inp, max_dec_inp)
sets['test'] = test_set.get_s2s_sets(word_indices, max_enc_inp, max_dec_inp)

In [11]:
print(sets['test'][0][0])
print(len(sets['test'][0][0]))
print(sets['test'][1][0])
print(len(sets['test'][1][0]))
print(sets['test'][2][0])
print(sets['test'][3][0])
print(sets['test'][4][0])
print(sets['test'][5][0])
print(sets['test'][6][0])
print(len(sets['test'][6][0]))

[42508, 2, 26523, 23549, 20418, 12155, 36588, 25755, 41240, 45523, 39154, 48730, 45523, 4560, 38419, 5702, 16969, 33093, 4560, 14886, 5702, 16969, 20418, 41595, 4560, 15287, 5702, 16969, 19084, 4560, 3690, 5702, 16969, 19084, 4560, 42508, 5702, 16969, 14636, 20418, 5702, 4560, 47344, 5702, 16969, 20418, 41595, 4560, 26054, 5702, 16969, 20418, 43717, 34798, 32693, 35388, 13376, 12646, 7917, 45125, 1943, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
97
[[1], [0], [2], [3], [4], [4], [0], [0], [2], [2], [0], [1], [0], [0], [1], [2], [2], [4], [0], [1], [2], [2], [4], [4], [0], [1], [2], [2], [4], [0], [1], [2], [2], [4], [0], [1], [2], [2], [3], [4], [4], [0], [1], [2], [2], [4], [4], [0], [1], [2], [2], [4], [4], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0

In [12]:
pred = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 1, 1, 2, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9]
sk.metrics.f1_score(sets['test'][4][0], pred, labels=[1, 2, 3, 4, 5, 6], average='micro')

TypeError: Expected sequence or array-like, got <class 'int'>

In [15]:
import tensorflow as tf
import numpy as np
import warnings
import matplotlib.pyplot as plt
import sklearn as sk

from Iterator import S2SIterator


class S2S_Model:

    def __init__(self, 
                 batch=50,
                 enc_vocab_size=100, 
                 dec_vocab_size=9, 
                 enc_emb_size=100, 
                 dec_emb_size=100, 
                 state_size=100, 
                 dropout=1.0,
                 learn_rate=0.001,
                 max_gradient_norm=5,
                 enc_emb_layer=False):
        self.batch = batch
        self.enc_vocab_size = enc_vocab_size
        self.dec_vocab_size = dec_vocab_size
        self.enc_emb_size = enc_emb_size
        self.dec_emb_size = dec_emb_size
        self.state_size = state_size
        self.dropout = dropout
        self.learn_rate = learn_rate
        self.max_gradient_norm=max_gradient_norm
        self.enc_emb_layer = enc_emb_layer
        self.graph = None
        self.sess = None

    def reset_graph(self):
        if self.sess:
            self.sess.close()
        tf.reset_default_graph()

    def build_graph(self):
        self.reset_graph()

        # Placeholders
        enc_x = tf.placeholder(name ='encoder_inputs', dtype=tf.int32, shape=[self.batch, None])  # [batch_size, num_steps]
        enc_labs = tf.placeholder(dtype=tf.float32, shape=[self.batch, None, 1])
        dec_x = tf.placeholder(name ='decoder_inputs', dtype=tf.int32, shape=[self.batch, None])
        y = tf.placeholder(name='target', dtype=tf.int32, shape=[self.batch, None])
        target_weights = tf.placeholder(tf.float32, shape=[self.batch, None])
        enc_seqlen = tf.placeholder(tf.int32, shape=[self.batch])
        dec_seqlen = tf.placeholder(tf.int32, shape=[self.batch])
        keep_prob = tf.placeholder(tf.float32)
        testing = tf.placeholder(tf.bool)
        
        # Embeddings
        if type(self.enc_emb_layer) != bool:
            enc_embeddings = tf.get_variable('enc_embedding_matrix', [self.enc_vocab_size, self.enc_emb_size], dtype=tf.float32)
        else:
            enc_embeddings = tf.get_variable('enc_embedding_matrix', initializer=tf.constant(self.enc_emb_layer))
        enc_inputs = tf.nn.embedding_lookup(enc_embeddings, enc_x)
        dec_inputs = tf.nn.embedding_lookup(enc_embeddings, dec_x)
        enc_inputs = tf.concat([enc_inputs, enc_labs], axis=-1)
        
        # Encoder
        
        # Singl Forward
        #encoder_cell = tf.nn.rnn_cell.BasicLSTMCell(self.state_size)
        
        #encoder_outputs, encoder_state = tf.nn.dynamic_rnn(cell=encoder_cell, 
        #                                                   inputs=enc_inputs,
        #                                                   sequence_length=enc_seqlen,
        #                                                   dtype=tf.float32)
        
        # Bidirectional
        forward_cell = tf.nn.rnn_cell.BasicLSTMCell(self.state_size)
        backward_cell = tf.nn.rnn_cell.BasicLSTMCell(self.state_size)

        bi_outputs, encoder_state = tf.nn.bidirectional_dynamic_rnn(forward_cell,
                                                                    backward_cell, 
                                                                    enc_inputs,
                                                                    sequence_length=enc_seqlen,
                                                                    dtype=tf.float32)
        
        # Conbining the output hidden states of ceells
        encoder_outputs = tf.concat(bi_outputs, -1)
        encoder_state = tf.concat(encoder_state, axis=-1)       
        encoder_state = tf.contrib.rnn.LSTMStateTuple(encoder_state[0], encoder_state[1])
        
        #Decoder
        decoder_cell = tf.nn.rnn_cell.BasicLSTMCell(2*self.state_size)

        # Attention
        attention_mechanism = tf.contrib.seq2seq.LuongAttention(2*self.state_size, 
                                                                encoder_outputs,
                                                                memory_sequence_length=enc_seqlen)
        
        decoder_cell = tf.contrib.seq2seq.AttentionWrapper(decoder_cell,
                                                           attention_mechanism,
                                                           attention_layer_size=self.state_size)
        
        attn_zero = decoder_cell.zero_state(batch_size=self.batch, dtype=tf.float32)
        encoder_state = attn_zero.clone(cell_state=encoder_state)
        
        # Helper
        helper = tf.contrib.seq2seq.TrainingHelper(dec_inputs, dec_seqlen)
        
        projection_layer = tf.contrib.keras.layers.Dense(self.dec_vocab_size, use_bias=False)
        decoder = tf.contrib.seq2seq.BasicDecoder(decoder_cell, 
                                                  helper, 
                                                  encoder_state,
                                                  output_layer=projection_layer)
        
        # Dynamic decoding
        outputs, _, _ = tf.contrib.seq2seq.dynamic_decode(decoder)
        logits = outputs.rnn_output
        
        crossent = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
        train_loss = (tf.reduce_sum(crossent * target_weights) / self.batch)
        wordnum = tf.to_float(tf.reduce_mean(dec_seqlen))
        ent_per_word = train_loss * self.batch / wordnum
        perplexity = tf.pow(tf.to_float(2), ent_per_word)
        
        # Calculate and clip gradients
        params = tf.trainable_variables()
        gradients = tf.gradients(train_loss, params)
        clipped_gradients, _ = tf.clip_by_global_norm(gradients, self.max_gradient_norm)
        
        # Optimization
        optimizer = tf.train.AdamOptimizer(self.learn_rate)
        update_step = optimizer.apply_gradients(zip(clipped_gradients, params))
        
        # Test Helper
        test_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(enc_embeddings,
                                                               tf.fill([tf.shape(enc_x)[0]], 3),
                                                               4)

        # Test Decoder
        test_decoder = tf.contrib.seq2seq.BasicDecoder(decoder_cell, 
                                                       test_helper, 
                                                       encoder_state,
                                                       output_layer=projection_layer)
        # Test Dynamic decoding
        maximum_iterations = tf.round(tf.reduce_max(enc_seqlen))
        test_outputs, _, _ = tf.contrib.seq2seq.dynamic_decode(test_decoder,
                                                               maximum_iterations=maximum_iterations)
        #prediction = test_outputs.sample_id
        prediction = tf.argmax(test_outputs.rnn_output, axis=2)

        self.graph = {'enc_x': enc_x,
                      'enc_labs': enc_labs,
                      'dec_x': dec_x,
                      'y': y,
                      'enc_seqlen': enc_seqlen,
                      'dec_seqlen': dec_seqlen,
                      'target_weights': target_weights,
                      'keep_prob': keep_prob,
                      'prediction': prediction,
                      'perplexity': perplexity,
                      'update_step': update_step
                      }

    def train(self, sets, epochs=10, report_percentage=1, show_progress=False, show_plot=False):
        # Start a tf session and run the optimisation algorithm
        self.sess = tf.Session()
        self.sess.run(tf.global_variables_initializer())

        trainer = S2SIterator(*(sets['train'][:-1]))

        train_truth = [truth[:end] for (truth, end) in zip(sets['train'][5], sets['train'][4])]
        validation_truth = [truth[:l] for (truth, l) in zip(sets['validation'][5], sets['validation'][4])]
        test_truth = [truth[:l] for (truth, l) in zip(sets['test'][5], sets['test'][4])]

        train_feed = {self.graph['enc_x']: sets['train'][0],
                      self.graph['enc_labs']: sets['train'][1],
                      self.graph['enc_seqlen']: sets['train'][2],
                      self.graph['dec_x']: sets['train'][3],
                      self.graph['dec_seqlen']: sets['train'][4],
                      self.graph['y']: sets['train'][5],
                      self.graph['target_weights']: sets['train'][6],
                      self.graph['keep_prob']: 1.0}
        validation_feed = {self.graph['enc_x']: sets['validation'][0],
                           self.graph['enc_labs']: sets['validation'][1],
                           self.graph['enc_seqlen']: sets['validation'][2],
                           self.graph['dec_x']: sets['validation'][3],
                           self.graph['dec_seqlen']: sets['validation'][4],
                           self.graph['y']: sets['validation'][5],
                           self.graph['target_weights']: sets['validation'][6],
                           self.graph['keep_prob']: 1.0}
        test_feed = {self.graph['enc_x']: sets['test'][0],
                     self.graph['enc_labs']: sets['test'][1],
                     self.graph['enc_seqlen']: sets['test'][2],
                     self.graph['dec_x']: sets['test'][3],
                     self.graph['dec_seqlen']: sets['test'][4],
                     self.graph['y']: sets['test'][5],
                     self.graph['target_weights']: sets['test'][6],
                     self.graph['keep_prob']: 1.0}

        train_f1_score = []
        validation_f1_score = []

        mark = (epochs * (len(sets['train'][0]) // self.batch) * report_percentage) // 100
        check_point = []
        N = 0

        warnings.simplefilter("ignore")
        while trainer.epochs < epochs:
            trex, trel, trexl, trdx, trdxl, trdy, trtw = trainer.next_batch(self.batch)
            feed = {self.graph['enc_x']: trex,
                    self.graph['enc_labs']: trel,
                    self.graph['enc_seqlen']: trexl, 
                    self.graph['dec_x']: trdx, 
                    self.graph['dec_seqlen']: trdxl,
                    self.graph['y']: trdy,
                    self.graph['target_weights']: trtw,
                    self.graph['keep_prob']: self.dropout}
            if N % mark == 0:
                #prediction = self.sess.run(self.graph['prediction'], feed_dict=train_feed)
                #pred_cut = [pred[:end] for (pred, end) in zip(prediction, sets['train'][3])]
                #f1_sum = 0
                #for i in range(len(pred_cut)):
                #    f1_sum += sk.metrics.f1_score(train_truth[i], pred_cut[i], labels=[1, 2, 3, 4, 5, 6], average='micro')       
                #train_f1_score.append(f1_sum / len(pred_cut))
                #prediction = self.sess.run(self.graph['prediction'], feed_dict=validation_feed)
                #pred_cut = [pred[:end] for (pred, end) in zip(prediction, sets['validation'][3])]
                #f1_sum = 0
                #for i in range(len(pred_cut)):
                #    f1_sum += sk.metrics.f1_score(validation_truth[i], pred_cut[i], labels=[1, 2, 3, 4, 5, 6], average='micro')       
                #validation_f1_score.append(f1_sum / len(pred_cut))
                #check_point.append(N)
                perp = self.sess.run(self.graph['perplexity'], feed_dict=feed)
                print('Perplexity: {}'.format(perp))
                if show_progress: print("Progress: %d%%" % (N * report_percentage // mark), end="\r")
            self.sess.run(self.graph['update_step'], feed_dict=feed)
            N += 1
        warnings.simplefilter("default")

        #test_prediction = self.sess.run(self.graph['prediction'], feed_dict=test_feed)
        #pred_cut = [pred[:end] for (pred, end) in zip(prediction, sets['test'][3])]
        #f1_sum = 0
        #for i in range(len(pred_cut)):
        #    f1_sum += sk.metrics.f1_score(test_truth[i], pred_cut[i], labels=[1, 2, 3, 4, 5, 6], average='micro')       
        #test_f1_score = f1_sum / len(pred_cut)
        
        #if show_progress:
        #    print('FInal Values: Tr-F1: {:.4f}, Val-F1: {:.4f}'.format(train_f1_score[-1], validation_f1_score[-1]))
        #    print("Test F1-Score: {:.4f}\n".format(test_f1_score))
        
        #if show_plot:
        #    np_check_point = np.array(check_point)
        #    np_train_f1 = np.array(train_f1_score)
        #    np_val_f1 = np.array(validation_f1_score)

        #    plt.plot(np_check_point, np_train_f1, label="Train")
        #    plt.plot(np_check_point, np_val_f1, label="Validation")
        #    plt.plot(np_check_point, np.ones(len(np_check_point))*0.35, label="Baseline")
        #    plt.xlabel("Batches")
        #    plt.ylabel("F1-Score")
        #    plt.legend()
        #    plt.show()
        
        #return train_f1_score, validation_f1_score, test_f1_score

    def predict(self, data):
        feed = {self.graph['enc_x']: data[0],
                self.graph['enc_labs']: data[1],
                self.graph['enc_seqlen']: data[2], 
                self.graph['dec_x']: data[3], 
                self.graph['dec_seqlen']: data[4],
                self.graph['y']: data[5],
                self.graph['target_weights']: data[6],
                self.graph['keep_prob']: 1.0}
        return self.sess.run(self.graph['prediction'], feed_dict=feed)

    def close(self):
        self.sess.close()

In [16]:
S2S = S2S_Model(batch=50,
                enc_vocab_size=len(word_indices), 
                dec_vocab_size=len(word_indices), 
                enc_emb_size=100, 
                dec_emb_size=100, 
                state_size=128, 
                dropout=1.0,
                learn_rate=0.0001,
                max_gradient_norm=5,
                enc_emb_layer=emb_layer)
S2S.build_graph()
S2S.train(sets=sets, epochs=1, report_percentage=10, show_progress=True, show_plot=True)

Perplexity: inf
Perplexity: inf
Perplexity: inf
Perplexity: inf
Perplexity: inf
Perplexity: inf
Perplexity: inf
Perplexity: inf
Perplexity: inf
Perplexity: inf
Perplexity: inf


In [18]:
tru_vocab = []
tru_words = sets['test'][7]
for i in range(len(sets['test'][5])):
    tru_vocab.append(sets['test'][5][i][:sets['test'][4][i]])

In [19]:
print(tru_vocab[:3])
print(tru_words[:3])

[[42508, 5702, 16969, 14636, 20418, 5702, 4], [48730, 33341, 5702, 26523, 23549, 19084, 4], [27732, 5702, 16969, 14636, 19084, 4]]
[['ofloxacin', '<num>', 'mg', 'po', 'q', '<num>', '<eos>'], ['insulin', 'nph', '<num>', 'units', 'subcu', 'bid', '<eos>'], ['colace', '<num>', 'mg', 'po', 'bid', '<eos>']]


In [22]:
res = []
for i in range(len(sets['test'][0])):
    dummy = [[], [], [], [], [], [], []]
    for j in range(50):
        for k in range(7):
            dummy[k].append(sets['test'][k][i])
    temp = S2S.predict(dummy)
    res.append(temp[0])

In [ ]:
res = []
for i in range(7):
    dummy = [[], [], [], [], [], []]
    for k in range(6):
        print('bla')
        dummy[k].append(sets['test'][k][i*50:(i+1)*50])
    temp = S2S.predict(dummy)
    res.append(temp)
dummy = [[], [], [], [], [], []]
for i in range(5):
    for k in range(6):
        dummy[k].append(sets['test'][k][350:])
temp = S2S.predict(dummy)
res.append(temp[:10])

In [23]:
res[:30]

[array([4], dtype=int64),
 array([4], dtype=int64),
 array([4], dtype=int64),
 array([4], dtype=int64),
 array([4], dtype=int64),
 array([4], dtype=int64),
 array([4], dtype=int64),
 array([4], dtype=int64),
 array([4], dtype=int64),
 array([4], dtype=int64),
 array([4], dtype=int64),
 array([4], dtype=int64),
 array([4], dtype=int64),
 array([4], dtype=int64),
 array([4], dtype=int64),
 array([4], dtype=int64),
 array([4], dtype=int64),
 array([4], dtype=int64),
 array([4], dtype=int64),
 array([4], dtype=int64),
 array([4], dtype=int64),
 array([4], dtype=int64),
 array([4], dtype=int64),
 array([4], dtype=int64),
 array([4], dtype=int64),
 array([4], dtype=int64),
 array([4], dtype=int64),
 array([4], dtype=int64),
 array([4], dtype=int64),
 array([4], dtype=int64)]

In [ ]:
for word in word_indices:
    if word_indices[word] == 40251:
        print(word)

In [ ]:
a = []
a.append(sets['test'][0][2:5])
print(len(a[0]))

In [ ]:
token_perf = fn.token_perf(res, tru)

In [ ]:
baseline = [0 if word in target_dict['reasons'] else 1 for word in sets['test_words']]
sk.metrics.f1_score(tru, baseline, pos_label=0, average='binary')

In [ ]:
fn.category_words(sets['test_words'], res, tru, 1, 0)

In [ ]:
fn.colour_text(sets['test_words'], res, tru)

In [ ]:
testers = Dataset.get_DS(stage='train', labelled='yes')
phrase_perf = fn.phrase_perf(target, RNN, testers, word_indices, side_words=[lw, rw], tfpn=True, show_phrases=True, case_info=True, rnn=True)

# HyperParameter Scan

In [ ]:
emb_sizes = [100]
emb_models = 1
target_saturations = [0.05, 0.1, 0.2, 0.5, 0.7]
layer_sizes = [50]
dropouts = [1.0]
learn_rates = [0.01]
epochs = [100]
NN_num = 5

case_num = len(emb_sizes)*emb_models*len(layer_sizes)*len(target_saturations)*len(epochs)*len(dropouts)*len(learn_rates)*NN_num
print(case_num)

In [ ]:
max_performance = 0
n = 1

for emb_size in emb_sizes:
    print('Model Number: %d/%d' %(n, case_num))
    for i in range(emb_models):
        model = Word2Vec(sentences, min_count=1, size=emb_size)
        for saturation in target_saturations:
            sets = fn.get_traintest2 (labelled_cases, model)
            fn.saturate_training_set(sets, model, target_dict['medications'], saturation)
            for layer_size in layer_sizes:
                for drop in dropouts:
                    for rate in learn_rates:
                        for epoch in epochs:
                            for j in range(NN_num):
                                print('Model Number: %d/%d' %(n, case_num))
                                print('ES: %d EM: %d sat: %f, LS: %d, drop: %f, LR: %f, epochs: %d, NN: %d' \
                                       % (emb_size, i, saturation, layer_size, drop, rate, epoch, j))
                                NN = FF_Model(input_size=emb_size, layers=[layer_size], dropout=drop, learn_rate=rate)
                                NN.build_graph()
                                NN.train(sets, epochs=epoch)
                                res = NN.predict(sets['test_set'])
                                tru = np.argmax(sets['test_labels'], 1)
                                perf = sk.metrics.f1_score(tru, res, pos_label=0)
                                if perf > max_performance:
                                    max_performance = perf
                                    NN.save_model('gold')
                                    model.save('gold/GOLDEMB')
                                NN.close()
                                n += 1